# Imports and global settings

In [ ]:
import json
import re
import math
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import FuncFormatter
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import FormatStrFormatter

ThreadFormatter = FormatStrFormatter('%.0f')

sns.set_style("whitegrid")
sns.set_context("paper", font_scale=1.5)
sns.set_palette("pastel")

tex_fonts = {
    # Use LaTeX to write all text
    "text.usetex": True,
    "font.family": "serif",
    # Use 11pt font in plots, to match 11pt font in document
    "axes.labelsize": 11,
    "font.size": 11,
    # Make the legend/label fonts a little smaller
    "legend.fontsize": 9,
    "legend.title_fontsize": 9,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "text.latex.preamble": "\\usepackage{latexsym}\\usepackage{amsmath}"
}

plt.rcParams.update(tex_fonts)
plt.rcParams['figure.figsize'] = (10, 5)

In [ ]:
def toGraphExponent(numNodes):
    return math.ceil(math.log2(numNodes))

def generateParallelKSPPlot(df):
    graphSize = df.iloc[0]['graphName']
    ax = sns.lineplot(data=df, x='ksp-threads', y='time', hue='algoName', palette='pastel', 
                      style="algoName", markers=True, dashes=False)
    ax.set(xlabel='Number of threads (deviations)', ylabel='runtime (seconds)',
           title=f"Parallel Runtime on $G(n,p)$ graph with $n\\approx 2^{{{20 + math.ceil(math.log2(graphSize))}}}$ and $p=\\frac{{4}}{{n}}$")
    plt.semilogx(base=2)
    plt.semilogy(base=2)
    ax.xaxis.set_major_formatter(ThreadFormatter)
    ax.yaxis.set_major_formatter(ScalarFormatter())
    plt.legend(title='Algorithms', ncol=2)
    plt.savefig(f'plots/parallel-ksp-runtime-{graphSize}.pdf')
    plt.show()
    
def generateParallelDSPlot(df):
    graphSize = df.iloc[0]['graphName']
    ax = sns.lineplot(data=df, x='ds-threads', y='time', hue='algoName', palette='pastel', 
                      style="algoName", markers=True, dashes=False)
    ax.set(xlabel='Number of threads ($\Delta$-Stepping)', ylabel='runtime (seconds)',
           title=f"Parallel Runtime on $G(n,p)$ graph with $n\\approx 2^{{{20 + math.ceil(math.log2(graphSize))}}}$ and $p=\\frac{{4}}{{n}}$")
    plt.semilogx(base=2)
    plt.semilogy(base=2)
    ax.xaxis.set_major_formatter(ThreadFormatter)
    ax.yaxis.set_major_formatter(ScalarFormatter())
    plt.legend(title='Algorithms', ncol=2)
    plt.savefig(f'plots/parallel-ds-runtime-{graphSize}.pdf')
    plt.show()
    
def getAlgoName(stats):
    baseName = stats['algo'].capitalize()
    if baseName == "Kim":
        return 'KIM'
        
    if stats['guided'] or stats['skipping'] > 0:
        baseName +=  '-'
        
    if stats['guided']:
        baseName += 'g'
        
    if stats['skipping'] > 0:
        baseName += 's'
        
    if stats['skipping'] == 2:
        baseName += '2'
        
    if 'skipByLength' in stats and stats['skipByLength']:
        baseName += '-l'
        
    # if 'earlyStopping' in stats and stats['earlyStopping']:
        # baseName += '-es'
    
    return baseName

def setLegendAlgoFont(axis, skip = [], replace = {}, ncols = 1):
    handles, labels = axis.get_legend_handles_labels()
    newlabels = ['\\textsc{' + l + '}' if i not in skip and i not in replace else (l if i not in replace else replace[i]) for i, l in enumerate(labels)]
    axis.legend(handles, newlabels, ncol=ncols)
    
def loadFilesIntoDataFrame(files):
    data = ''
    for f in files:
        file = f'../data/runtimes/{f}.json'
        print(f'Loading file "{file}"')
        with open(file, 'r') as file:
            data = data + file.read().replace('\n', '')
    
    obj = json.loads("[" + data[:-1] + "]")
    
    for i, item in enumerate(obj):
        # print(item)
        nodeExp = toGraphExponent(obj[i]['numNodes'])
        obj[i]['run'] = obj[i]['run'] + 1
        obj[i]['algoName'] = getAlgoName(obj[i])
        obj[i]['graphExp'] = nodeExp
        obj[i]['graphName'] = 2**(nodeExp-20)
        obj[i]['graphName2'] = f'$2^{{{nodeExp}}}$'
        obj[i]['numNodes2'] = 2**nodeExp
        # obj[i]['time_n'] = obj[i]['time'] / (obj[i]['numNodes'] / (2**20))
        obj[i]['time_n'] = obj[i]['time'] / (2**(nodeExp-20))
        obj[i]['time_n2'] = obj[i]['time'] / (2**(nodeExp-25) * nodeExp)
        obj[i]['threadCode'] = f"{obj[i]['kspThreads']}-{obj[i]['dsThreads']}"
        obj[i]['file'] = obj[i]['file'] if "scratch" not in obj[i]['file'] else re.search('graphs/([^./]+)\.metis', obj[i]['file']).group(1)
    
    return pd.DataFrame(obj)

def finalizePlot(fileName=None):
    plt.tight_layout()
    if fileName is not None:
        plt.savefig(fileName, bbox_inches='tight')
    plt.show()

algosAllColors = []
algosAllColors += sns.color_palette("light:#f7546f", n_colors=2)[1:2]     # Yen without anything 
algosAllColors += sns.color_palette("light:#b143c4", n_colors=2)[1:2]     # Yen without guiding
algosAllColors += sns.color_palette("light:#4162cc", n_colors=3)[1:3]     # Yen with skipping, without guiding
algosAllColors += sns.color_palette("light:#3da84d", n_colors=3)[1:3]
algosAllColors += algosAllColors[2:6]                                     # Yen with skipping and guiding
algosAllColors += algosAllColors[2:6]                                     # Feng
algosAllColors += sns.color_palette("light:#cc7218", n_colors=2)[1:2]
# sns.color_palette(algosAllColors)
    
algosAll = ['Yen', 'Yen-g', 'Yen-s', 'Yen-s2', 'Yen-s-l', 'Yen-s2-l', 'Yen-gs', 'Yen-gs2', 'Yen-gs-l', 'Yen-gs2-l', 'Feng-gs', 'Feng-gs2', 'Feng-gs-l', 'Feng-gs2-l', 'KIM']

def getAlgoColors(algoList):
    
    colors = []
    for algo in algoList:
        colors.append(algosAllColors[algosAll.index(algo)])
        
    return sns.color_palette(colors)
        

algosBasic = ['Yen', 'Yen-g', 'Yen-s', 'Yen-gs', 'Feng-gs']

gnp4 = ['gnp4-1','gnp4-2','gnp4-4','gnp4-8','gnp4-16','gnp4-32','gnp4-64','gnp4-128','gnp4-256']
gnp4Small = ['gnp4-1','gnp4-2','gnp4-4','gnp4-8','gnp4-16','gnp4-32']
gnp64 = ['gnp64-1','gnp64-2','gnp64-4','gnp64-8','gnp64-16','gnp64-32','gnp64-64']
grid = ['grid-8-10','grid-9-11','grid-10-12','grid-11-13','grid-12-14']
udGnp = ['ud-' + g for g in gnp4]
udGrid = ['ud-' + g for g in grid]

In [ ]:
runtimes = {}

for f in ['1.1.1-64', '1.1-64.1', '64.1-64.1']:
    file = f'../data/runtimes/{f}.json'
    print(f'Loading file "{file}"')
    with open(file, 'r') as file:
        runtimes[f] = json.loads("[" + file.read().replace('\n', '')[:-1] + "]")

dfs = {}

for experiment in runtimes:
    for i in range(0, len(runtimes[experiment])):  
        runtimes[experiment][i]['graphExp'] = toGraphExponent(runtimes[experiment][i]['numNodes'])
        runtimes[experiment][i]['graphName'] = 2**(toGraphExponent(runtimes[experiment][i]['numNodes'])-20)
        runtimes[experiment][i]['count'] = 1
        runtimes[experiment][i]['algoName'] = getAlgoName(runtimes[experiment][i])
    
    dfs[experiment] = pd.DataFrame(runtimes[experiment])
    print(dfs[experiment] \
        .filter(items=['algoName', 'graphName', 'count', 'ksp-threads', 'ds-threads']) \
        .groupby(by=["algoName", 'graphName']) \
        .count().to_string())


# Parallel Runningtimes KSP only

In [ ]:
generateParallelKSPPlot(dfs['1.1-64.1'])
generateParallelKSPPlot(dfs['64.1-64.1'])

# Parallel Runningtime DS only

In [ ]:
generateParallelDSPlot(dfs['1.1.1-64'])

# Load All Cluster Runtimes

In [ ]:
# load all runtimes
runtimes = loadFilesIntoDataFrame(['cluster']).drop_duplicates(["algoName", "file", "run", 'earlyStopping'])
parallel = loadFilesIntoDataFrame(['parallel']).drop_duplicates(["algoName", "file", "run", 'threadCode'])
undirected = loadFilesIntoDataFrame(['undirected']).drop_duplicates(["algoName", "file", "run", 'threadCode'])
realworld = loadFilesIntoDataFrame(['realworld']).drop_duplicates(["algoName", "file", "run", 'threadCode'])

# split up runtimes
sequential = runtimes[(runtimes['earlyStopping'] == True) & (runtimes['kspThreads'] == 1) & (runtimes['dsThreads'] == 1)]
earlyStopping = runtimes[(runtimes['earlyStopping'] == False) & (runtimes['kspThreads'] == 1) & (runtimes['dsThreads'] == 1)]
parallelKSP = parallel[parallel['dsThreads'] == 1]
parallelDS = parallel[parallel['kspThreads'] == 1]

# show colormap of number of experiments
# cm = sns.diverging_palette(10, 133, as_cmap=True)
cm = sns.cubehelix_palette(start=-0.85, rot=-0.8, dark=0.2, light=0.65, gamma=0.7, as_cmap=True, hue=1.7, reverse=False)
# cm = sns.color_palette("coolwarm", as_cmap=True)
ax = sns.heatmap(sequential.groupby(["algoName", "file"]).sum()['run'].unstack(level=0), annot=True, fmt="n", cmap=cm, linewidths=0.1, vmin=0, vmax=210)
ax.set(title="Sequential")
plt.tight_layout()
plt.show()

ax = sns.heatmap(earlyStopping.groupby(["algoName", "file"]).sum()['run'].unstack(level=0), annot=True, fmt="n", cmap=cm, linewidths=0.05, vmin=0, vmax=210)
ax.set(title="Early Stopping")
plt.tight_layout()
plt.show()

ax = sns.heatmap(parallelDS.groupby(["algoName", "threadCode"]).sum()['run'].unstack(level=0), annot=True, fmt="n", cmap=cm, linewidths=0.05, vmin=0, vmax=210)
ax.set(title="Parallel DS")
plt.tight_layout()
plt.show()

ax = sns.heatmap(parallelKSP.groupby(["algoName", "threadCode"]).sum()['run'].unstack(level=0), annot=True, fmt="n", cmap=cm, linewidths=0.05, vmin=0, vmax=210)
ax.set(title="Parallel KSP")
plt.tight_layout()
plt.show()

ax = sns.heatmap(undirected.groupby(["algoName", "file"]).sum()['run'].unstack(level=0), annot=True, fmt="n", cmap=cm, linewidths=0.05, vmin=0, vmax=210)
ax.set(title="Undirected")
plt.tight_layout()
plt.show()

ax = sns.heatmap(realworld.groupby(["algoName", "file"]).sum()['run'].unstack(level=0), annot=True, fmt="n", cmap=cm, linewidths=0.05, vmin=0, vmax=465)
ax.set(title="Real World")
plt.tight_layout()
plt.show()

dataFilter = realworld.algoName.isin(['Yen-gs2']) & realworld.file.isin(['orkut.prod']) 
realworld[dataFilter]['run']

# runtimes[(runtimes['earlyStopping'] == True) & (runtimes['kspThreads'] == 1) & (runtimes['dsThreads'] == 1)].groupby(["algoName", "file"]).sum()['run'].unstack(level=0)

# Early Stopping

In [ ]:
# filter data
earlyStopping = runtimes.algoName.isin(algosBasic) & runtimes.file.isin(gnp4Small) 

# draw plot
ax = sns.lineplot(data=runtimes[earlyStopping], x='numNodes2', y='time', hue='algoName', hue_order=algosBasic,
                  style='earlyStopping', palette='muted')
plt.xscale('log', base=2)
plt.yscale('log')
ax.set(xlabel='$n$ of $\\mathcal{G}(n,\\frac{4}{n})$ graph', ylabel='runtime (seconds)',
           title="Performance comparison with and without stopping SSSP computations early")
setLegendAlgoFont(ax, [7, 8], {0:'Algorithm', 6:'Early Stopping'}, 3)

finalizePlot('plots/performance/early-stopping-gnp4.pdf')

# Sequential Performance

In [ ]:
# filter data
algoStyles = {'Yen': 'solid', 
              'Yen-g': 'dashed', 
              'Yen-s': 'dotted', 'Yen-s2': 'dotted', 'Yen-s-l': 'dotted', 'Yen-s2-l': 'dotted', 
              'Yen-gs': 'dashed', 'Yen-gs2': 'dashed', 'Yen-gs-l': 'dashed', 'Yen-gs2-l': 'dashed', 
              'Feng-gs': 'solid', 'Feng-gs-l': 'solid', 'Feng-gs2': 'solid', 'Feng-gs2-l': 'solid',
              'KIM': 'solid'}

algosBestGnp = ['Yen-s', 'Yen-s2',  'Yen-s-l',  'Yen-s2-l', 'Yen-gs', 'Yen-gs2', 'Yen-gs-l', 'Yen-gs2-l']
algosBestGrid = ['Yen-s-l',  'Yen-s2-l', 'Yen-gs-l', 'Yen-gs2-l', 'Feng-gs-l', 'Feng-gs2-l']
# algoGroups=[algosAll, 
#             ['Yen-g',  'Yen-gs', 'Yen-gs2', 'Yen-gs-l', 'Yen-gs2-l', 'Yen-s', 'Yen-s2',  'Yen-s-l',  'Yen-s2-l'],
#             ['Feng-gs', 'Feng-gs2', 'Feng-gs-l', 'Feng-gs2-l']]
# for i, algos in enumerate(algoGroups):
for g in [['$\\mathcal{G}(n,\\frac{4}{n})$', 'gnp4', gnp4, algosBestGnp], 
          ['$\\mathcal{G}(n,\\frac{64}{n})$', 'gnp64', gnp64, algosBestGnp], 
          ['$\\operatorname{Grid}(n,4,0.8)$', 'grid', grid, algosBestGrid]]:
    
    dataFilter = runtimes.earlyStopping.eq(True) & runtimes.file.isin(g[2])
    minExp = runtimes[dataFilter]['graphExp'].min()
    maxExp = runtimes[dataFilter]['graphExp'].max()
    ticks = [2**x for x in range(minExp, maxExp + 1)]
    
    fig, (a0, a1) = plt.subplots(1, 2, gridspec_kw={'width_ratios': [1, 1]}, figsize=(10, 5))

    # Full 
    colors = getAlgoColors(algosAll).as_hex()
    for j, algo in enumerate(algosAll):
        dataFilter = runtimes.earlyStopping.eq(True) & runtimes.algoName.isin([algo]) & runtimes.file.isin(g[2])

        # draw plot
        ax = sns.lineplot(data=runtimes[dataFilter], x='numNodes2', y='time', color=colors[j], linestyle=algoStyles[algo], ci=None, label=algo, ax=a0)

    xLabel = f'$n$ of {g[0]} graph'
        
    a0.set_xscale('log', base=2)
    a0.set_yscale('log')
    a0.set(xlabel=xLabel, ylabel='runtime (seconds)', xticks=(ticks))
    
    a0.yaxis.set_major_locator(ticker.LogLocator(base=10, numticks=12) )
    a0.yaxis.set_minor_locator(ticker.LogLocator(base=10, subs=(0.2,0.4,0.6,0.8), numticks=12))
    a0.tick_params(which="both", left=True, color='gray', width=0.8)
    
    setLegendAlgoFont(a0, [], {}, 2)

    # Best
    colors = getAlgoColors(g[3]).as_hex()
    for j, algo in enumerate(g[3]):
        dataFilter = runtimes.earlyStopping.eq(True) & runtimes.algoName.isin([algo]) & runtimes.file.isin(g[2])

        # draw plot
        sns.lineplot(data=runtimes[dataFilter], x='numNodes2', y='time_n2', color=colors[j], linestyle=algoStyles[algo], ci=None, ax=a1)

    a1.set_xscale('log', base=2)
    # plt.yscale('log', base=2)
    
    a1.set(xlabel=xLabel, ylabel='runtime / $\\frac{n \\log n}{2^{15}}$ (seconds)', xticks=(ticks))
    plt.suptitle(f"Sequential Performance Comparison on {g[0]} Graphs", fontsize=16)

    finalizePlot(f'plots/performance/sequential-{g[1]}.pdf')

## Undirected

In [ ]:

runtimes = undirected

dataFilterGnp = runtimes.file.isin(udGnp)
dataFilterGrid = runtimes.file.isin(udGrid)
fig, (a0, a1) = plt.subplots(1, 2, gridspec_kw={'width_ratios': [1, 1]}, figsize=(10, 5))

for (plot, graphs, graphName) in [[a0, udGnp, '$\\mathcal{G}(n,\\frac{4}{n})$'], [a1, udGrid, '$\\operatorname{Grid}(n,4,0.8)$']]:
    minExp = runtimes[runtimes.file.isin(graphs)]['graphExp'].min()
    maxExp = runtimes[runtimes.file.isin(graphs)]['graphExp'].max()
    ticks = [2**x for x in range(minExp, maxExp + 1)]

    # Gnp 
    colors = getAlgoColors(algosAll).as_hex()
    for j, algo in enumerate(algosAll):
        dataFilter = runtimes.algoName.isin([algo]) & runtimes.file.isin(graphs)

        # draw plot
        sns.lineplot(data=runtimes[dataFilter], x='numNodes2', y='time', color=colors[j], linestyle=algoStyles[algo], ci=None, label=algo, ax=plot)

    plot.set_xscale('log', base=2)
    plot.set_yscale('log')
    # a0.set(xlabel=f'$n$ of $\\mathcal{G}(n,\\frac{4}{n})$ graph', ylabel='runtime (seconds)', xticks=(ticks))
    plot.set(xlabel=f'$n$ of {graphName} graph', ylabel='runtime (seconds)', xticks=(ticks))
    
    plot.yaxis.set_major_locator(ticker.LogLocator(base=10, numticks=12) )
    plot.yaxis.set_minor_locator(ticker.LogLocator(base=10, subs=(0.2,0.4,0.6,0.8), numticks=12))
    plot.tick_params(which="both", left=True, color='gray', width=0.8)


# a1.set(xlabel='$n$ of Grid($n,\\frac{4}{1}$) graph', ylabel='runtime (seconds)', xticks=(ticks))
setLegendAlgoFont(a0, [], {}, 2)
a1.get_legend().remove()

# a1.set(xlabel=xLabel, ylabel='runtime / $\\frac{n \\log n}{2^{15}}$ (seconds)', xticks=(ticks))
plt.suptitle(f"Sequential Performance Comparison on Undirected Graphs", fontsize=16)

finalizePlot(f'plots/performance/undirected.pdf')

# Parallel Performance

In [ ]:
algosParallel = ['Yen-gs', 'Yen-gs2', 'Yen-gs-l', 'Yen-gs2-l', 'Feng-gs', 'Feng-gs2', 'Feng-gs-l', 'Feng-gs2-l']

colors = getAlgoColors(algosParallel).as_hex()

algoStyles = {'Yen': 'solid', 
              'Yen-g': 'dashed', 
              'Yen-s': 'dotted', 'Yen-s2': 'dotted', 'Yen-s-l': 'dotted', 'Yen-s2-l': 'dotted', 
              'Yen-gs': 'dashed', 'Yen-gs2': 'dashed', 'Yen-gs-l': 'dashed', 'Yen-gs2-l': 'dashed', 
              'Feng-gs': 'solid', 'Feng-gs-l': 'solid', 'Feng-gs2': 'solid', 'Feng-gs2-l': 'solid'}

# fig, (a0, a1) = plt.subplots(1, 2, gridspec_kw={'width_ratios': [1, 1]}, figsize=(10, 5))
fig, (a0, a1) = plt.subplots(1, 2, sharey=True, gridspec_kw={'width_ratios': [5, 4]}, figsize=(10, 5))

# parallel kSP
for j, algo in enumerate(algosParallel):
    dataFilter = parallelKSP.algoName.isin([algo])

    # draw plot
    sns.lineplot(data=parallelKSP[dataFilter], x='kspThreads', y='time', color=colors[j], linestyle=algoStyles[algo], ci=None, label=algo, ax=a0)

# a0.set_xscale('log', base=2)
# a0.set_yscale('log', base=2)
a0.set(xlabel='Number of parallel deviation threads', ylabel='runtime (seconds)')
# ax.set(xlabel=xLabel, ylabel='runtime (seconds)', xticks=(ticks))

a0.axvline(x=20, color='gray', linestyle='dotted')

setLegendAlgoFont(a0, [], {}, 2)

# parallel DS
for j, algo in enumerate(algosParallel):
    dataFilter = parallelDS.algoName.isin([algo])

    # draw plot
    sns.lineplot(data=parallelDS[dataFilter], x='dsThreads', y='time', color=colors[j], linestyle=algoStyles[algo], ci=None, label=algo, ax=a1)

# a1.set_xscale('log', base=2)
# a1.set_yscale('log', base=2)
a1.set(xlabel='Number of $\Delta$-stepping threads')
# ax.set(xlabel=xLabel, ylabel='runtime (seconds)', xticks=(ticks))

# a1.yaxis.set_major_locator(ticker.LogLocator(base=2, numticks=12) )
# a1.yaxis.set_minor_locator(ticker.LogLocator(base=2, subs=(0.25,0.5,0.75), numticks=12))
# a1.tick_params(which="both", left=True, color='gray', width=0.8)

a0.set_xscale('log', base=2)
a1.set_xscale('log', base=2)
a0.set_yscale('log', base=2)
a0.yaxis.set_major_locator(ticker.LogLocator(base=2, numticks=12) )
a0.yaxis.set_minor_locator(ticker.LogLocator(base=2, subs=(0.25,0.5,0.75), numticks=12))
a0.tick_params(which="both", left=True, color='gray', width=0.8)
a1.yaxis.set_major_locator(ticker.LogLocator(base=2, numticks=12) )
a1.yaxis.set_minor_locator(ticker.LogLocator(base=2, subs=(0.25,0.5,0.75), numticks=12))
a1.tick_params(which="both", left=True, color='gray', width=0.8)

a1.get_legend().remove()
# a1.set(xlabel=xLabel, ylabel='runtime / $\\frac{n \\log n}{2^{15}}$ (seconds)', xticks=(ticks))
plt.suptitle("Parallel Performance on $\\operatorname{Grid}(2^{26},4,0.8)$ Graphs", fontsize=16)

finalizePlot('plots/performance/parallel-grid.pdf')

# Real World Graphs

In [ ]:
algoColors = getAlgoColors(algosAll[1:]).as_hex()

algos = ['Yen-g', 'Yen-s', 'Yen-s2', 'Yen-s-l', 'Yen-s2-l', 'Yen-gs', 'Yen-gs2', 'Yen-gs-l', 'Yen-gs2-l', 'Feng-gs', 'Feng-gs2', 'Feng-gs-l', 'Feng-gs2-l']

ax = sns.catplot(y="time", x="algoName", col="file", kind="box", height=3, aspect=5/3, col_wrap=2, sharey=False, sharex=True, palette=algoColors,
                 order=algos,
                 data=realworld[(realworld["file"].str.contains("orkut"))])
# ax.set_titles(["unweighted", "random weights ob $[0,1]$", "degree sum", "degree product", "inverse degree sum", "inverse degree product"])
ax.set_axis_labels("", "runtime (seconds)")
ax.axes[0].set_title("unweighted")
ax.axes[1].set_title("uniform random weights over $[0,1]$")
ax.axes[2].set_title("degree sum")
ax.axes[3].set_title("degree product")
ax.axes[4].set_title("inverse degree sum")
ax.axes[5].set_title("inverse degree product")

# ax.axes[0].set_yscale('log', base=2)
ax.axes[1].set_yscale('log', base=2)
ax.axes[2].set_yscale('log', base=2)
ax.axes[3].set_yscale('log', base=2)

ax.axes[4].tick_params(axis='x', rotation=45)
ax.axes[5].tick_params(axis='x', rotation=45)
ax.axes[5].set_xticklabels([f"\\textsc{{{x}}}" for x in algos])
plt.suptitle(f"Orkut network", fontsize=16)
finalizePlot('plots/performance/rw-orkut.pdf')


ax = sns.catplot(y="time", x="algoName", col="file", kind="box", height=5, aspect=5/5, col_wrap=2, sharey=True, sharex=True, palette=algoColors,
                 order=algos,
                 data=realworld[realworld["file"].str.contains("scc-eur")])
ax.set_axis_labels("", "runtime (seconds)")
ax.axes[0].set_title("distance")
ax.axes[1].set_title("travel time")
ax.axes[0].set_yscale('log', base=2)
ax.axes[1].set_yscale('log', base=2)
ax.axes[0].tick_params(axis='x', rotation=45)
ax.axes[1].tick_params(axis='x', rotation=45)
ax.axes[1].set_xticklabels([f"\\textsc{{{x}}}" for x in algos])
plt.suptitle(f"European road network", fontsize=16)
finalizePlot('plots/performance/rw-euroroad.pdf')

In [ ]:
def showFactor(RTD, slowAlgo, fastAlgo):
    # filter1 = runtimes.earlyStopping.eq(True) & runtimes.file.isin(gnp4)
    meanTimes = RTD[RTD.algoName.isin([slowAlgo, fastAlgo])][['algoName','file','time']] \
                .groupby(['file','algoName']).mean() \
                .pivot_table(index='file', columns='algoName')['time']
    # print(RTD)
    meanTimes['factor'] = meanTimes[slowAlgo] / meanTimes[fastAlgo]
    print(meanTimes,'\n')
    print('Min: ', meanTimes['factor'].min(),
          '\nMean: ', meanTimes['factor'].mean(),
          '\nMedian: ', meanTimes['factor'].median(),
          '\nMax: ', meanTimes['factor'].max(),'\n\n\n')
    
def showFactor2(RTD, slowAlgo, fastAlgo):
    # filter1 = runtimes.earlyStopping.eq(True) & runtimes.file.isin(gnp4)
    meanTimes = RTD[RTD.algoName.isin([slowAlgo, fastAlgo])][['algoName','file','time']] \
                .groupby(['file','algoName']).median() \
                .pivot_table(index='file', columns='algoName')['time']
    # print(RTD)
    meanTimes['factor'] = meanTimes[slowAlgo] / meanTimes[fastAlgo]
    print(meanTimes,'\n')
    
# sequential[sequential.file.isin(gnp4)]
# showFactor(sequential[sequential.file.isin(gnp4)], 'Yen-s', 'Yen-gs2-l')

# showFactor(sequential[sequential.file.isin(grid)], 'Yen', 'Yen-g')
# showFactor(sequential[sequential.file.isin(grid)], 'Yen-s', 'Yen-g')
# showFactor(sequential[sequential.file.isin(grid)], 'Yen-s2', 'Yen-g')

grid2 = grid[-2:]

# showFactor(sequential[sequential.file.isin(grid2)], 'Yen-s', 'Yen-s-l')
# showFactor(sequential[sequential.file.isin(grid2)], 'Yen-s2', 'Yen-s2-l')

# showFactor(sequential[sequential.file.isin(grid2)], 'Yen-gs', 'Yen-gs-l')
# showFactor(sequential[sequential.file.isin(grid2)], 'Yen-gs2', 'Yen-gs2-l')

# showFactor(sequential[sequential.file.isin(grid2)], 'Feng-gs', 'Feng-gs-l')
# showFactor(sequential[sequential.file.isin(grid2)], 'Feng-gs2', 'Feng-gs2-l')

# showFactor(sequential[sequential.file.isin(grid)], 'Feng-gs', 'Yen-gs-l')
# showFactor(sequential[sequential.file.isin(grid)], 'Feng-gs', 'Yen-gs2-l')

# showFactor(undirected[undirected.file.isin(udGnp)], 'KIM', 'Yen-gs2-l')
# showFactor(undirected[undirected.file.isin(udGrid)], 'KIM', 'Yen-gs2-l')


# showFactor(sequential[sequential.file.isin(grid2)], 'Feng-gs2-l', 'Yen-gs2-l')


# showFactor2(realworld[realworld["file"].str.contains("scc-eur")], 'Yen-s', 'Yen-s-l')
# showFactor2(realworld[realworld["file"].str.contains("scc-eur")], 'Yen-s2', 'Yen-s2-l')
# showFactor2(realworld[realworld["file"].str.contains("scc-eur")], 'Yen-gs', 'Yen-gs-l')
# showFactor2(realworld[realworld["file"].str.contains("scc-eur")], 'Yen-gs2', 'Yen-gs2-l')
# showFactor2(realworld[realworld["file"].str.contains("scc-eur")], 'Feng-gs', 'Feng-gs-l')
# showFactor2(realworld[realworld["file"].str.contains("scc-eur")], 'Feng-gs2', 'Feng-gs2-l')

showFactor2(realworld[realworld["file"].str.contains("scc-eur")], 'Yen-gs2-l', 'Feng-gs2-l')

# showFactor2(realworld[realworld["file"].str.contains("scc-eur")], 'Yen-s2', 'Feng-gs-l')
# showFactor2(realworld[realworld["file"].str.contains("scc-eur")], 'Yen-s2', 'Feng-gs2-l')